## Bokeh Chloropleth
- Using .json file created from ESRI ABS shapefiles

In [1]:
shape_path = "/users/danielcorcoran/desktop/lga_esri_16/lga16vic_simplified/lga_vic_16_simplified.shp"

In [2]:
import fiona 
shp = fiona.open(shape_path)

In [8]:
# Extract features from shapefile
district_name = [ feat["properties"]["LGA_NAME16"] for feat in shp if feat['geometry'] is not None]
district_area = [ feat["properties"]["AREASQKM16"] for feat in shp if feat['geometry'] is not None]
lga_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]
lga_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]

In [10]:
''' Imports '''
import bokeh
from bokeh.plotting import figure, show, output_file
import numpy
from bokeh.models import HoverTool, ColumnDataSource,LogColorMapper


''' Tools '''
TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"


''' Data '''
possible_colours = ['#edf8fb','#b3cde3','#8c96c6','#8856a7','#810f7c']
possible_categories = ['very low', 'low', 'medium', 'high', 'very high']
test_colors = []
test_categories = []

for n in range(len(district_x)):
    random_index = numpy.random.randint(0, len(possible_colours))
    test_colors.append(possible_colours[random_index])
    test_categories.append(possible_categories[random_index])
    
color_map = LogColorMapper(palette = possible_colours)

random_value1 = [numpy.random.uniform(0,10) for n in range(len(district_x))]    
random_value2 = [numpy.random.uniform(123,549) for n in range(len(district_x))]

source = ColumnDataSource(
    data=dict(x=district_x, 
              y=district_y,
             value1 = random_value1,
             value2 = random_value2,
             lganame = district_name,
             lgaarea = district_area,
             colourcode = test_colors,
             categories = test_categories))


''' Create figure '''
p = figure(
    title="Local Government Areas (2016) Bokeh Plot",
    toolbar_location="right",
    plot_width=1100,
    plot_height=800,
    tools=TOOLS)


''' Patches '''
p.patches(
    'x',
    'y',
    source=source,
    fill_alpha=1,
    fill_color = 'colourcode',
    #fill_color={'field': 'value1', 'transform': color_map},
    line_color = 'white', 
    line_width = 0.2,
legend = 'categories')


''' Remove grid lines '''
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.legend.location = "top_right"

''' Set Hover Tool '''
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [('Value 1', '@value1'),
                  ('Value 2','@value2'),
                  ('Longitude, Latitude', '$x,$y'),
                 ('LGA Name', '@lganame'),
                 ('LGA Area', '@lgaarea')]


''' Output file optional'''
#output_file("choropleth.html", title="choropleth.py example")

show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: cols, x, y [renderer: GlyphRenderer(id='09fdea51-be51-4b83-8b3e-c8978cb90a66', ...)]


In [13]:
from bokeh.io import show, output_notebook, output_file
from bokeh.models import (
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper
)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6

with open('/users/danielcorcoran/desktop/lga_esri_16/LGA16_SIMPLIFIED.geojson', 'r') as f:
    geo_source = GeoJSONDataSource(geojson=f.read())


TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Title", tools=TOOLS, x_axis_location=None, y_axis_location=None, width=1200, height=800)
p.grid.grid_line_color = None

p.patches('xs', 'ys', fill_alpha=1, fill_color = 'orange',
          line_color='white', line_width=0.5, source=geo_source)


# hover = p.select_one(HoverTool)
# hover.point_policy = "follow_mouse"
# hover.tooltips = [("Provincia:", "@provincia")]

# output_file("PBIar.html", title="Testing islands in bokeh")

show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: cols, x, y [renderer: GlyphRenderer(id='09fdea51-be51-4b83-8b3e-c8978cb90a66', ...)]
